<h2>Project 1: $k$-Nearest Neighbors</h2>
<p><cite><center>So many points,<br>
some near some far,<br>
- who are my true neighbors?</center></cite></p>

<h3>Introduction</h3>

<p>In this project, you will build a $k$-nearest neighbor classifier.</p>

<strong>How to submit:</strong> You can submit your code using the blue <strong>Submit</strong> button above. This button will send any code below surrounded by <strong>#&lt;GRADED&gt;</strong><strong>#&lt;/GRADED&gt;</strong> tags below to the autograder, which will then run several tests over your code. By clicking on the <strong>Details</strong> dropdown next to the Submit button, you will be able to view your submission report once the autograder has completed running. This submission report contains a summary of the tests you have failed or passed, as well as a log of any errors generated by your code when we ran it.

Note that this may take a while depending on how long your code takes to run! Once your code is submitted you may navigate away from the page as you desire -- the most recent submission report will always be available from the Details menu.

<p><strong>Evaluation:</strong> Your code will be autograded for technical
correctness and--on some assignments--speed. Please <em>do not</em> change the names of any provided functions or classes within the code, or you will wreak havoc on the autograder. Furthermore, <em>any code not surrounded by <strong>#&lt;GRADED&gt;</strong><strong>#&lt;/GRADED&gt;</strong> tags will not be run by the autograder</em>. However, the correctness of your implementation -- not the autograder's output -- will be the final judge of your score.  If necessary, we will review and grade assignments individually to ensure that you receive due credit for your work.

<p><strong>Academic Integrity:</strong> We will be checking your code against other submissions in the class for logical redundancy. If you copy someone else's code and submit it with minor changes, we will know. These cheat detectors are quite hard to fool, so please don't try. We trust you all to submit your own work only; <em>please</em> don't let us down. If you do, we will pursue the strongest consequences available to us.

<p><strong>Getting Help:</strong> You are not alone!  If you find yourself stuck  on something, contact the course staff for help.  Office hours, section, and the <a href="https://piazza.com/class/icxgflcnpra3ko">Piazza</a> are there for your support; please use them.  If you can't make our office hours, let us know and we will schedule more.  We want these projects to be rewarding and instructional, not frustrating and demoralizing.  But, we don't know when or how to help unless you ask.  



**Libraries**: Before we get started we need to install a few libraries. You can do this by executing the following code.

In [60]:
#<GRADED>
import numpy as np
# functions that may be helpful
from scipy.stats import mode
#</GRADED>
import sys
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
from scipy.io import loadmat
import time
from helper_functions import loaddata, visualize_knn_2D, visualize_knn_images, plotfaces

print('You\'re running python %s' % sys.version.split(' ')[0])

You're running python 3.6.4


<h3> k-Nearest Neighbors implementation in Python </h3>

<p>Our goal towards a $k$NN classifier is to build a classifier for face recognition. 
</p>

**Data:** We first obtain some data for testing your code. The data resides in the files <code>faces.mat</code> which hold the dataset for further experiments.

Here, <b>xTr</b> are the training vectors with labels <b>yTr</b> and <b>xTe</b> are the testing vectors with labels <b>yTe</b>. 
As a reminder, to predict the label or class of an image in <b>xTe</b>, we will look for the <i>k</i>-nearest neighbors in <b>xTr</b> and predict a label based on their labels in <b>yTr</b>. For evaluation, we will compare these labels against the true labels provided in <b>yTe</b>.</p>

<h4> Visualizing data</h4>

Let us take a look at our data. The following script will take the first 10 training images from the face data set and visualize them.

In [61]:
xTr,yTr,xTe,yTe=loaddata("faces.mat")

plt.figure()
plotfaces(xTr[:9, :])

<IPython.core.display.Javascript object>


<h4> Implementation </h4>
<p> The following questions will ask you to finish these functions in a pre-defined order. <br></p>

<p>(a) Implement the function  <b><code>l2distance</code></b>. You may use your own code(s) from the previous project.</p>


In [62]:
#<GRADED>
import scipy
def l2distance(X,Z=None):
    """
    function D=l2distance(X,Z)
    
    Computes the Euclidean distance matrix.
    Syntax:
    D=l2distance(X,Z)
    Input:
    X: nxd data matrix with n vectors (rows) of dimensionality d
    Z: mxd data matrix with m vectors (rows) of dimensionality d
    
    Output:
    Matrix D of size nxm
    D(i,j) is the Euclidean distance of X(i,:) and Z(j,:)
    
    call with only one input:
    l2distance(X)=l2distance(X,X)
    """

#      if Z is None: # case when there is only one input (X)
#         Z=X;
    
#     # your code goes here ...
#     Z = np.transpose(Z)
#     p = np.dot(X,Z)
#     print(p)
#     return p

    
    if Z is None:
        Z=X;

    n,d1=X.shape
    m,d2=Z.shape
    assert (d1==d2), "Dimensions of input vectors must match!"
    Xt = np.transpose(X)
    ipX = np.dot(X,Xt)
    S = np.diag(ipX)
    S = np.tile(S,(m,1))
    S = np.transpose(S)
    Zt = np.transpose(Z)
    ipZ = np.dot(Z,Zt)
    R = np.diag(ipZ)
    R = np.tile(R,(n,1))
    G = np.dot(X,Zt)
    D = np.sqrt(S+R-(2*G))
    return D
    
    # ... until here
#</GRADED>


<p>(b) Implement the function <b><code>findknn</code></b>, which should find the $k$ nearest neighbors of a set of vectors within a given training data set. The call of 
<pre>
 [I,D]=findknn(xTr,xTe,k);
</pre> 
should result in two matrices $I$ and $D$, both of dimensions $k\times n$, where $n$ is the number of input vectors in <code>xTe</code>. The matrix $I(i,j)$ is the index of the $i^{th}$ nearest neighbor of the vector $xTe(j,:)$. 
So, for example, if we set <code>i=I(1,3)</code>, then <code>xTr(i,:)</code> is the first nearest neighbor of vector <code>xTe(3,:)</code>. The second matrix $D$ returns the corresponding distances. So $D(i,j)$ is the distance of $xTe(j,:)$ to its $i^{th}$ nearest neighbor.
</p>

In [63]:
#<GRADED>

def findknn(xTr,xTe,k):
    """
    function [indices,dists]=findknn(xTr,xTe,k);
    
    Finds the k nearest neighbors of xTe in xTr.
    
    Input:
    xTr = nxd input matrix with n row-vectors of dimensionality d
    xTe = mxd input matrix with m row-vectors of dimensionality d
    k = number of nearest neighbors to be found
    
    Output:
    indices = kxm matrix, where indices(i,j) is the i^th nearest neighbor of xTe(j,:)
    dists = Euclidean distances to the respective nearest neighbors
    """
    # Enter your code here
    D = l2distance(xTr,xTe)
    I = np.argsort(D,axis=0)
    
    sor = np.sort(D,axis=0)
    
    dists = sor[:k]
    
    
    indices = I[:k]

    return indices,dists
    # until here

#</GRADED>

<p> The following demo samples random points in 2D. If your findknn  function is correctly implemented, you should be able to click anywhere on the plot to add a test point. The function should then draw direct connections from your test point to the k  nearest neighbors. Verify manually if your code is correct.
</p>

In [64]:
visualize_knn_2D(findknn)

<IPython.core.display.Javascript object>

We can visualize the k=3 nearest training neighbors of some of the test points (Click on the image to cycle through different test points). 

In [65]:
visualize_knn_images(findknn, imageType='faces')

<IPython.core.display.Javascript object>

Click on the images above, to cycle through the test images.


/usr/local/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


<p>(c) The function <b><code>analyze</code></b> should compute various metrics to evaluate a classifier. The call of
<pre>
  result=analyze(kind,truth,preds);
</pre>
should output the <b>accuracy</b> or <b>absolute loss</b> in variable <code>result</code>. The type of output required can be specified in the input argument <code>kind</code> as <code>"abs"</code> or <code>"acc"</code>. The input variables <code>truth</code> and <code>pred</code> should contain vectors of true and predicted labels respectively.
For example, the call
<pre>
>> analyze('acc',[1 2 1 2],[1 2 1 1])
</pre>
should return an accuracy of 0.75. Here, the true labels are 1,2,1,2 and the predicted labels are 1,2,1,1. So the first three examples are classified correctly, and the last one is wrong --- 75% accuracy.
</p>



In [66]:
#<GRADED>
def analyze(kind,truth,preds):
    """
    function output=analyze(kind,truth,preds)         
    Analyses the accuracy of a prediction
    Input:
    kind='acc' classification error
    kind='abs' absolute loss
    (other values of 'kind' will follow later)
    """
    
    truth = truth.flatten()
    preds = preds.flatten()
    
    if kind == 'abs':
        # compute the absolute difference between truth and predictions
#         output = np.absolute(np.array(truth) - np.array(preds))
#         output = (np.sum(np.absolute(truth - preds)))/len(truth)
        temp = np.absolute(truth - preds)
        counter = 0
        for i in range(len(temp)):
            counter = counter + temp[i]
        counter = counter/len(truth)
        return counter
    elif kind == 'acc':
        output = 0
        for i in range(len(truth)):
            if truth[i] == preds[i]:
                output += 1
        output = output/len(truth)
        return output
                    
    

            

#</GRADED>


<p>(d) Implement the function <b><code>knnclassifier</code></b>, which should perform $k$ nearest neighbor classification on a given test data set. The call <pre>preds=knnclassifier(xTr,yTr,xTe,k)</pre>
should output the predictions for the data in <code>xTe</code> i.e. <code>preds[i]</code> will contain the prediction for <code>xTe[i,:]</code>.</p>

In [93]:
#<GRADED>
def knnclassifier(xTr,yTr,xTe,k):
    """
    function preds=knnclassifier(xTr,yTr,xTe,k);
    
    k-nn classifier 
    
    Input:
    xTr = nxd input matrix with n row-vectors of dimensionality d
    xTe = mxd input matrix with m row-vectors of dimensionality d
    k = number of nearest neighbors to be found
    
    Output:
    
    preds = predicted labels, ie preds(i) is the predicted label of xTe(i,:)
    """
    # fix array shapes
    yTr = yTr.flatten()
    i,d = findknn(xTr,xTe,k)
    #print(i)
    it = i#.transpose()
    # set of labels
    pr0 = yTr[it]
    a,b = mode(pr0)
    #print(a)
    return a.flatten()
#     print("this is predes 0")
#     print(preds_0)
#     print("this is a and b") 
#     print(a)
#     print(b)
#     print("this is preds final")
#     print(preds)
    #call mode on ou
#     for i in range(0,m):
        
#         for j in range(0,k):
    # preds is mx1 vector containing predictions for xTe(i,:) 
    

#</GRADED>

<p>You can compute the actual classification error on the test set by calling
<pre>
>> analyze("acc",yTe,knnclassifier(xTr,yTr,xTe,3))
</pre></p>

<p>(e) This script runs the $k$-nearest neighbor classifier over the faces data set. The faces data set has $40$ classes. What classification accuracy would you expect from a random classifier?</p>

In [92]:
print("Face Recognition: (1-nn)")
xTr,yTr,xTe,yTe=loaddata("faces.mat") # load the data
t0 = time.time()
preds = knnclassifier(xTr,yTr,xTe,1)
result=analyze("acc",yTe,preds)
t1 = time.time()
print("You obtained %.2f%% classification acccuracy in %.4f seconds\n" % (result*100.0,t1-t0))

Face Recognition: (1-nn)
[[ 89 261 263 210 275 203 216 232 151 159 200 154 256 248 232 172 206 227
   28  15 169 144 224  73  53 253 177  43 123  54  42  51 118 261 149 268
   82  66 138  32 142 273 240 256  92   8 160 101 158 158 233  53 108 107
    4  19 134 214  87 279 187 216 133 106  79 216  63 262 163 225  21 226
  109 233 150 135  54 252  29 201 148  76 133 119 117 223 224  21 109 168
  102 222 210 231  10 147  81 267 234 143 163  15   6 117  53 173 239 171
  112  28  42 252 132 100 146  80   7 118 120 157]]
[[15  3 14 29  4 18  5 33 13 40  2  6  4 34 33 17 31 12 24 25  6  5  1  5
  20 14  9  1 29 18  8 33 21  3 39 32 39 26  3  7 22 35 37  4 10 13 19 25
  28 28 37 20 19  6 14 19  9 16 11 26 35  5 32 16 24  5 30  8 27 16 38 15
   2 37 36 23 18 22 17  7 23 12 32 13 11  3  1 38  2 23 35 12 29 27 26 39
   7 30 34 15 27 25 10 11 20 21 31 36 17 24  8 22 23 34 28 30 38 21 31  9]]
You obtained 95.83% classification acccuracy in 0.0316 seconds



<p>(f) (optional) Sometimes a $k$-NN classifier can result in a draw, when the majority vote is not clearly defined. Can you improve your accuracy by falling back onto $k$-NN with lower $k$ in such a case?</p>
